In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')
from importlib import reload

# Import OpenAI key
from helper import get_openai_api_key
openai_api_key = get_openai_api_key()

In [2]:
from openai import OpenAI
from IPython.display import display, Markdown
import pandas as pd
import sqlite3
from concurrent.futures import ThreadPoolExecutor
import functionDefinitions
import csv
import json
import os,sys
import matplotlib.pyplot as plt

GPT_MODEL = 'gpt-4o-mini'
O1_MODEL = 'o1-mini'

client = OpenAI(api_key=openai_api_key)

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [3]:
reload(functionDefinitions)
conn = sqlite3.connect(r'FlightBooking.db') 
TOOLS = functionDefinitions.BookingClass(conn).get_booking_function_tools()

In [4]:
## Utility function to highlight difference between two texts
from redlines import Redlines
from IPython.display import display, Markdown

def compare_strings(string1, string2):
    # Create a Redlines object to compare the strings
    diff = Redlines(string1, string2)
    
    # Display the differences using Markdown
    display(Markdown(diff.output_markdown))

# Example usage
string_a = "This is the original text for comparison purposes."
string_b = "This is the modified text to compare for differences."

compare_strings(string_a, string_b)

This is the <span style='color:red;font-weight:700;text-decoration:line-through;'>original </span><span style='color:green;font-weight:700;'>modified </span>text <span style='color:green;font-weight:700;'>to compare </span>for <span style='color:red;font-weight:700;text-decoration:line-through;'>comparison purposes.</span><span style='color:green;font-weight:700;'>differences.</span>

## Step 1: Generate 4o routine (prompt)

We'll take the Flight Cancellation Policy that we have created and convert it to an LLM-based routine with the following prompt.

This process is also a good use of O1, using a purely text document to create instructions and tools for an LLM to make use of. To dive deeper on this topic check out [this cookbook](https://cookbook.openai.com/examples/o1/using_reasoning_for_routine_generation).

#### Read the Flight Cancellation Policy from the Original Policy folder

In [5]:
with open('originalPolicy/flightCancellationsPolicy.md', 'r') as file:
    flight_cancellation_policy = file.read()
    print(flight_cancellation_policy)

# **Internal Flight Cancellations and Changes Policy**

**Purpose**: This document serves as a detailed guide for internal support agents to handle flight cancellations and changes. The focus is on providing clear instructions, ensuring efficiency, consistency, and customer satisfaction during calls.

**Note**: Always maintain a calm, empathetic tone while assisting customers, especially during stressful situations involving cancellations or major changes.

## **Table of Contents**

1. General Guidelines for Handling Customer Requests  
2. Cancellations: Types and Policies  
3. Changes: Types and Policies  
4. Rebooking Guidelines  
5. Compensation and Refund Rules  
6. Special Cases  
7. FAQs for Common Scenarios

### **1\. General Guidelines for Handling Customer Requests**

* **Confirm Identity**: Verify the customer's identity by asking for their booking reference and any additional required details (e.g., name and flight number).  
* **Listen and Understand**: Clarify if the custo

#### Prompt to generate the instruction set from the policy document
#### TOOLS contain the various functions that will need to be called in the instruction set to execute the instruction

In [6]:
CONVERSION_PROMPT = f"""
You are a helpful assistant tasked with taking an external facing help center article and converting it into a internal-facing programmatically executable routine optimized for an LLM. 
The LLM using this routine will be tasked with reading the policy, answering incoming questions from customers, and helping drive the case toward resolution. Use the markdown format.

Please follow these instructions:
1. **Review the customer service policy carefully** to ensure every step is accounted for. It is crucial not to skip any steps or policies.
2. **Organize the instructions into a logical, step-by-step order**, using the specified format. 
3. **Use the following format**:
   - **Main actions are numbered** (e.g., 1, 2, 3).
   - **Sub-actions are lettered** under their relevant main actions (e.g., 1a, 1b).
      **Sub-actions should start on new lines**
   - **Specify conditions using clear 'if...then...else' statements** (e.g., 'If the product was purchased within 30 days, then...').
   - **For instructions that require more information from the customer**, provide polite and professional prompts to ask for additional information.
   - **For actions that require data from external systems**, write a step to call a function using backticks for the function name (e.g., call the `check_delivery_date` function).
      - **If a step requires the customer service agent to take an action** (e.g., process a refund), generate a function call for this action (e.g., call the `process_refund` function).
      - **Only use the available set of functions that are defined below.
   - **If there is an action an assistant can perform on behalf of the user**, include a function call for this action (e.g., call the `change_email_address` function), and ensure the function is defined with its purpose and required parameters.
      - **Only use the available set of functions that are defined below.
   - **The step prior to case resolution should always be to ask if there is anything more you can assist with**.
   - **End with a final action for case resolution**: calling the `case_resolution` function should always be the final step.
4. **Ensure compliance** by making sure all steps adhere to company policies, privacy regulations, and legal requirements.
5. **Handle exceptions or escalations** by specifying steps for scenarios that fall outside the standard policy.
6. **Ensure coverage** by checking that all of the conditions covered in the policy are also covered in the routines

**Important**: Always wrap the functions you return in backticks i.e. `check_ticket_type`. Do not include the arguments to the functions.

Here are the currently available set of functions in JSON format: 
TOOLS: {TOOLS}

Please convert the following customer service policy into the formatted routine, ensuring it is easy to follow and execute programmatically. Ensure that you **only** use the functions provided and do **not** create net new functions."""

#### Generate the flight cancellation instruction set routine

In [7]:
def generate_routine(policy):
    try:
        messages = [
            {
                "role": "user",
                "content": f"""
                    {CONVERSION_PROMPT}

                    POLICY:
                    {policy}
                """
            }
        ]

        response = client.chat.completions.create(
            model=f"""{O1_MODEL}""",
            messages=messages
        )
        

        return response.choices[0].message.content 
    except Exception as e:
        print(f"An error occurred: {e}")

In [1]:
## This statement can be uncommented the first time you run this notebook to see the auto generated version from o1 model
## Since the model results are probabilitic, you will see slightly different results each time
# flight_cancellation_routine = generate_routine(flight_cancellation_policy)

In [12]:
# Open the file in read mode
# file = open("evals/flight_cancellation_routine.txt", "r")
# flight_cancellation_routine = str(file.read())

In [7]:
#flight_cancellation_routine = '```markdown\n1. **Confirm Identity**\n   a. Prompt the customer to provide their booking reference, full name, and flight number.\n      - Call the `verify_identity` function.\n\n2. **Listen and Understand**\n   a. Ask the customer to clarify if they wish to cancel, change, or inquire about compensation.\n      - Call the `ask_clarification` function with the prompt: "Could you please clarify if you would like to cancel your booking, make changes to your flight, or inquire about compensation?"\n\n3. **Check the Ticket Type**\n   a. Retrieve the ticket type based on the booking reference.\n      - Call the `check_ticket_type` function.\n\n4. **Handle Cancellations**\n   a. If the customer initiated the cancellation:\n      i. If the ticket is refundable:\n         - If the cancellation is within 24 hours of booking, then:\n            1. Process a full refund.\n               - Call the `process_full_refund` function.\n         - Else:\n            1. Check specific fare rules for applicable refund amounts.\n               - Call the `check_fare_rules` function.\n            2. If a partial refund is applicable, then:\n               - Process a partial refund.\n                  - Call the `process_partial_refund` function.\n      ii. Else if the ticket is non-refundable:\n         - Offer flight credit for future use.\n            - Call the `offer_flight_credit` function.\n         - Inform the customer about any applicable penalty fees.\n   b. Else if the airline initiated the cancellation:\n      i. If the cancellation is weather-related:\n         - Rebook the customer on the next available flight without fees.\n            - Call the `rebook_without_fee` function.\n         - If rebooking is not acceptable, then:\n            - Process a full refund.\n               - Call the `process_full_refund` function.\n      ii. Else if the cancellation is due to mechanical issues or operational changes:\n         - Prioritize rebooking for the customer.\n            - Call the `prioritize_missed_connections` function.\n         - If an overnight stay is required, offer hotel accommodation.\n            - Call the `offer_accommodation` function.\n   c. Else if the customer is a no-show:\n      i. If the ticket is non-refundable, then:\n         - Inform the customer that no refunds are available but offer travel credits if applicable.\n            - Call the `offer_flight_credit` function.\n      ii. Else if the ticket is refundable, then:\n         - Refund applicable taxes or airport fees.\n            - Call the `process_partial_refund` function.\n\n5. **Handle Changes**\n   a. If the customer initiated the change:\n      i. If the change is same-day:\n         - If the ticket is flexible, then:\n            1. Allow changes without fees, subject to availability.\n               - Call the `process_change_no_fee` function.\n         - Else:\n            1. Charge a change fee.\n            2. Apply any fare differences.\n               - Call the `apply_change_fee` function.\n      ii. Else if the change is in advance:\n         - If the change is within 7 days of departure, then:\n            1. Apply the standard change fee.\n               - Call the `apply_change_fee` function.\n         - Else if the change is beyond 7 days, then:\n            1. Apply a lesser or no change fee based on ticket type.\n               - Call the `apply_lesser_change_fee` function.\n   b. Else if the airline initiated the change:\n      i. If the schedule change is minor (less than 2 hours), then:\n         - Inform the customer of the new schedule.\n         - Offer an alternative flight upon request.\n            - Call the `provide_alternative_options` function.\n      ii. Else if the schedule change is major (2+ hours), then:\n         - Provide options to rebook, receive a full refund, or obtain travel credit.\n            - Call the `rebook_without_fee` function or `process_full_refund` function accordingly.\n\n6. **Rebooking Guidelines**\n   a. Prioritize rebooking the customer on the next available flight operated by our airline.\n      - Call the `check_next_available_flight` function.\n      - If available, then:\n         - Book the flight.\n            - Call the `book_flight` function.\n   b. If no suitable options are available, then:\n      - Check interline partners for alternative connections.\n         - Call the `check_interline_partners` function.\n      - If an interline partner flight is found, then:\n         - Book the interline flight.\n            - Call the `book_interline_flight` function.\n   c. If the preferred flight is fully booked, then:\n      - Offer the next available option.\n      - Place the customer on a waitlist for their preferred flight if possible.\n      - Offer an upgrade to a higher class if available.\n         - Call the `offer_available_upgrades` function.\n\n7. **Compensation and Refund Rules**\n   a. **Refund Processing**\n      i. Inform the customer about the refund timeline based on the payment method.\n      ii. If the refund is to the original payment method, then:\n         - Process the refund.\n            - Call the `process_refund` function.\n      iii. Else, offer travel credits with a bonus percentage.\n         - Apply the travel credit.\n            - Call the `apply_travel_credit` function.\n   b. **Compensation**\n      i. Determine eligibility for compensation based on regional rules.\n         - Call the `check_compensation_eligibility` function.\n      ii. If eligible for meals or accommodation due to delays exceeding four hours, then:\n         - Provide meal vouchers or hotel accommodation.\n            - Call the `offer_accommodation` function.\n\n8. **Special Cases**\n   a. **Medical Emergencies**\n      i. If a medical certificate is provided, then:\n         - Allow full cancellation or offer flight credit.\n            - Call the `process_flexible_cancellation` function.\n      ii. If the customer or immediate family member is ill, then:\n         - Waive change fees.\n            - Call the `waive_change_fees` function.\n   b. **Bereavement Cases**\n      i. Offer flexibility on cancellations or changes.\n      ii. Request proper bereavement documentation.\n         - Call the `apply_bereavement_flexibility` function.\n   c. **Group Bookings**\n      i. Allow individual cancellations within a group booking.\n         - Call the `process_partial_group_cancellation` function.\n      ii. Permit one name change per passenger at no additional cost if made 7+ days before departure.\n         - Call the `permit_name_change` function.\n   d. **Unaccompanied Minors**\n      i. Ensure proper supervision is arranged for flight changes.\n         - Call the `arrange_supervision` function.\n      ii. Prioritize rebooking for unaccompanied minors on the next available flight.\n         - Call the `prioritize_minors_rebooking` function.\n\n9. **FAQs for Common Scenarios**\n   a. **Connecting Flight Affected**\n      - Always prioritize rebooking for missed connections caused by airline delays.\n         - Call the `prioritize_missed_connections` function.\n   b. **Handling Upgrade Requests After Cancellation**\n      - Offer upgrades based on availability.\n         - Call the `offer_available_upgrades` function.\n      - If unavailable, advise on fare differences.\n   c. **Changing Destination**\n      - Allow changes to a different destination.\n      - Advise on applicable fare differences and change fees.\n         - Call the `process_destination_change` function.\n   d. **Third-Party Bookings**\n      - Request the customer to contact the original booking agent.\n      - Provide assistance if eligible for direct airline handling.\n         - Call the `assist_third_party_booking` function.\n   e. **Compensation for Inconvenience**\n      - Assess eligibility based on the airline\'s compensation policy.\n         - Call the `assess_compensation_request` function.\n      - Inform the customer about available compensation options.\n   f. **Preferred Flight Fully Booked**\n      - Offer the next available option.\n      - Place the customer on a waitlist for their preferred flight.\n      - Offer an upgrade to a higher class if available.\n         - Call the `offer_available_upgrades` function.\n\n10. **Offer Further Assistance**\n    a. Ask the customer if there is anything more you can assist with.\n       - Call the `ask_clarification` function with the prompt: "Is there anything else I can assist you with today?"\n\n11. **Case Resolution**\n    a. Finalize and close the customer case with all resolution details.\n       - Call the `case_resolution` function.\n```'

##### This is the updated flight cancellation routine from the auto generated one from o1 model.  Not much of a change.  Added a statement about checking fare rules in the Cancellation section.
##### Also in the Handle changes, added a statement to check for the next available flight before processing the change.

In [8]:
flight_cancellation_routine = '```markdown\n1. **Confirm Identity**\n   a. Prompt the customer to provide their booking reference, full name, and flight number.\n      - Call the `verify_identity` function.\n\n2. **Listen and Understand**\n   a. Ask the customer to clarify if they wish to cancel, change, or inquire about compensation.\n      - Call the `ask_clarification` function with the prompt: "Could you please clarify if you would like to cancel your booking, make changes to your flight, or inquire about compensation?"\n\n3. **Check the Ticket Type**\n   a. Retrieve the ticket type based on the booking reference.\n      - Call the `check_ticket_type` function.\n\n4. **Handle Cancellations**\n   a. If the customer initiated the cancellation:\n  i. If the ticket is refundable:\n - Check specific fare rules for applicable refund amounts.  \n - Call the `check_fare_rules` function.\n    1.  If a full refund is applicable, then:\n Process a full refund.\n - Call the `process_full_refund` function.\n  - Else:\n                        2. If a partial refund is applicable, then:\n               - Process a partial refund.\n                  - Call the `process_partial_refund` function.\n      ii. Else if the ticket is non-refundable:\n         - Offer flight credit for future use.\n            - Call the `offer_flight_credit` function.\n         - Inform the customer about any applicable penalty fees.\n   b. Else if the airline initiated the cancellation:\n      i. If the cancellation is weather-related:\n         - Rebook the customer on the next available flight without fees.\n            - Call the `rebook_without_fee` function.\n         - If rebooking is not acceptable, then:\n            - Process a full refund.\n               - Call the `process_full_refund` function.\n      ii. Else if the cancellation is due to mechanical issues or operational changes:\n         - Prioritize rebooking for the customer.\n            - Call the `prioritize_missed_connections` function.\n         - If an overnight stay is required, offer hotel accommodation.\n            - Call the `offer_accommodation` function.\n   c. Else if the customer is a no-show:\n      i. If the ticket is non-refundable, then:\n         - Inform the customer that no refunds are available but offer travel credits if applicable.\n            - Call the `offer_flight_credit` function.\n      ii. Else if the ticket is refundable, then:\n         - Refund applicable taxes or airport fees.\n            - Call the `process_partial_refund` function.\n\n5. **Handle Changes**\n  a. If the customer initiated the change:\n      i. If the change is same-day:\n     - Rebook the customer on the next available flight by calling the `check_next_available_flight` function\n     - If the ticket is flexible, then:\n            1. Allow changes without fees, subject to availability.\n               - Call the `process_change_no_fee` function.\n         - Else:\n            1. Charge a change fee.\n            2. Apply any fare differences.\n               - Call the `apply_change_fee` function.\n      ii. Else if the change is in advance:\n      - Rebook the customer by calling the `provide_alternative_options` function \n      - If the change is within 7 days of departure, then:\n            1. Apply the standard change fee.\n               - Call the `apply_change_fee` function.\n         - Else if the change is beyond 7 days, then:\n            1. Apply a lesser or no change fee based on ticket type.\n               - Call the `apply_lesser_change_fee` function.\n   b. Else if the airline initiated the change:\n      i. If the schedule change is minor (less than 2 hours), then:\n         - Inform the customer of the new schedule.\n         - Offer an alternative flight upon request.\n            - Call the `provide_alternative_options` function.\n      ii. Else if the schedule change is major (2+ hours), then:\n         - Provide options to rebook, receive a full refund, or obtain travel credit.\n            - Call the `rebook_without_fee` function or `process_full_refund` function accordingly.\n\n6. **Rebooking or Change Guidelines**\n   a. Prioritize rebooking the customer on the next available flight operated by our airline.\n      - Call the `check_next_available_flight` function.\n      - If available, then:\n         - Book the flight.\n            - Call the `book_flight` function.\n   b. If no suitable options are available, then:\n      - Check interline partners for alternative connections.\n         - Call the `check_interline_partners` function.\n      - If an interline partner flight is found, then:\n         - Book the interline flight.\n            - Call the `book_interline_flight` function.\n   c. If the preferred flight is fully booked, then:\n      - Offer the next available option.\n      - Place the customer on a waitlist for their preferred flight if possible.\n      - Offer an upgrade to a higher class if available.\n         - Call the `offer_available_upgrades` function.\n\n7. **Compensation and Refund Rules**\n   a. **Refund Processing**\n      i. Inform the customer about the refund timeline based on the payment method.\n      ii. If the refund is to the original payment method, then:\n         - Process the refund.\n            - Call the `process_refund` function.\n      iii. Else, offer travel credits with a bonus percentage.\n         - Apply the travel credit.\n            - Call the `apply_travel_credit` function.\n   b. **Compensation**\n      i. Determine eligibility for compensation based on regional rules.\n         - Call the `check_compensation_eligibility` function.\n      ii. If eligible for meals or accommodation due to delays exceeding four hours, then:\n         - Provide meal vouchers or hotel accommodation.\n            - Call the `offer_accommodation` function.\n\n8. **Special Cases**\n   a. **Medical Emergencies**\n    Ask the Customer for a medical certificate. \n    i. If a medical certificate is provided, then:\n         - Allow full cancellation or offer flight credit.\n            - Call the `process_flexible_cancellation` function.\n      ii. If the customer or immediate family member is ill, then:\n         - Waive change fees.\n            - Call the `waive_change_fees` function.\n   b. **Bereavement Cases**\n      i. Offer flexibility on cancellations or changes.\n      ii. Request proper bereavement documentation.\n         - Call the `apply_bereavement_flexibility` function.\n   c. **Group Bookings**\n      i. Allow individual cancellations within a group booking.\n         - Call the `process_partial_group_cancellation` function.\n      ii. Permit one name change per passenger at no additional cost if made 7+ days before departure.\n         - Call the `permit_name_change` function.\n   d. **Unaccompanied Minors**\n      i. Ensure proper supervision is arranged for flight changes.\n         - Call the `arrange_supervision` function.\n      ii. Prioritize rebooking for unaccompanied minors on the next available flight.\n         - Call the `prioritize_minors_rebooking` function.\n\n9. **FAQs for Common Scenarios**\n   a. **Connecting Flight Affected**\n      - Always prioritize rebooking for missed connections caused by airline delays.\n         - Call the `prioritize_missed_connections` function.\n   b. **Handling Upgrade Requests After Cancellation**\n      - Offer upgrades based on availability.\n         - Call the `offer_available_upgrades` function.\n      - If unavailable, advise on fare differences.\n   c. **Changing Destination**\n      - Allow changes to a different destination.\n      - Advise on applicable fare differences and change fees.\n         - Call the `process_destination_change` function.\n   d. **Third-Party Bookings**\n      - Request the customer to contact the original booking agent.\n      - Provide assistance if eligible for direct airline handling.\n         - Call the `assist_third_party_booking` function.\n   e. **Compensation for Inconvenience**\n      - Assess eligibility based on the airline\'s compensation policy.\n         - Call the `assess_compensation_request` function.\n      - Inform the customer about available compensation options.\n   f. **Preferred Flight Fully Booked**\n      - Offer the next available option.\n      - Place the customer on a waitlist for their preferred flight.\n      - Offer an upgrade to a higher class if available.\n         - Call the `offer_available_upgrades` function.\n\n10. **Offer Further Assistance**\n    a. Ask the customer if there is anything more you can assist with.\n       - Call the `ask_clarification` function with the prompt: "Is there anything else I can assist you with today?"\n\n11. **Case Resolution**\n    a. Finalize and close the customer case with all resolution details.\n       - Call the `case_resolution` function.\n```'

In [9]:
display(Markdown(flight_cancellation_routine))

```markdown
1. **Confirm Identity**
   a. Prompt the customer to provide their booking reference, full name, and flight number.
      - Call the `verify_identity` function.

2. **Listen and Understand**
   a. Ask the customer to clarify if they wish to cancel, change, or inquire about compensation.
      - Call the `ask_clarification` function with the prompt: "Could you please clarify if you would like to cancel your booking, make changes to your flight, or inquire about compensation?"

3. **Check the Ticket Type**
   a. Retrieve the ticket type based on the booking reference.
      - Call the `check_ticket_type` function.

4. **Handle Cancellations**
   a. If the customer initiated the cancellation:
  i. If the ticket is refundable:
 - Check specific fare rules for applicable refund amounts.  
 - Call the `check_fare_rules` function.
    1.  If a full refund is applicable, then:
 Process a full refund.
 - Call the `process_full_refund` function.
  - Else:
                        2. If a partial refund is applicable, then:
               - Process a partial refund.
                  - Call the `process_partial_refund` function.
      ii. Else if the ticket is non-refundable:
         - Offer flight credit for future use.
            - Call the `offer_flight_credit` function.
         - Inform the customer about any applicable penalty fees.
   b. Else if the airline initiated the cancellation:
      i. If the cancellation is weather-related:
         - Rebook the customer on the next available flight without fees.
            - Call the `rebook_without_fee` function.
         - If rebooking is not acceptable, then:
            - Process a full refund.
               - Call the `process_full_refund` function.
      ii. Else if the cancellation is due to mechanical issues or operational changes:
         - Prioritize rebooking for the customer.
            - Call the `prioritize_missed_connections` function.
         - If an overnight stay is required, offer hotel accommodation.
            - Call the `offer_accommodation` function.
   c. Else if the customer is a no-show:
      i. If the ticket is non-refundable, then:
         - Inform the customer that no refunds are available but offer travel credits if applicable.
            - Call the `offer_flight_credit` function.
      ii. Else if the ticket is refundable, then:
         - Refund applicable taxes or airport fees.
            - Call the `process_partial_refund` function.

5. **Handle Changes**
  a. If the customer initiated the change:
      i. If the change is same-day:
     - Rebook the customer on the next available flight by calling the `check_next_available_flight` function
     - If the ticket is flexible, then:
            1. Allow changes without fees, subject to availability.
               - Call the `process_change_no_fee` function.
         - Else:
            1. Charge a change fee.
            2. Apply any fare differences.
               - Call the `apply_change_fee` function.
      ii. Else if the change is in advance:
      - Rebook the customer by calling the `provide_alternative_options` function 
      - If the change is within 7 days of departure, then:
            1. Apply the standard change fee.
               - Call the `apply_change_fee` function.
         - Else if the change is beyond 7 days, then:
            1. Apply a lesser or no change fee based on ticket type.
               - Call the `apply_lesser_change_fee` function.
   b. Else if the airline initiated the change:
      i. If the schedule change is minor (less than 2 hours), then:
         - Inform the customer of the new schedule.
         - Offer an alternative flight upon request.
            - Call the `provide_alternative_options` function.
      ii. Else if the schedule change is major (2+ hours), then:
         - Provide options to rebook, receive a full refund, or obtain travel credit.
            - Call the `rebook_without_fee` function or `process_full_refund` function accordingly.

6. **Rebooking or Change Guidelines**
   a. Prioritize rebooking the customer on the next available flight operated by our airline.
      - Call the `check_next_available_flight` function.
      - If available, then:
         - Book the flight.
            - Call the `book_flight` function.
   b. If no suitable options are available, then:
      - Check interline partners for alternative connections.
         - Call the `check_interline_partners` function.
      - If an interline partner flight is found, then:
         - Book the interline flight.
            - Call the `book_interline_flight` function.
   c. If the preferred flight is fully booked, then:
      - Offer the next available option.
      - Place the customer on a waitlist for their preferred flight if possible.
      - Offer an upgrade to a higher class if available.
         - Call the `offer_available_upgrades` function.

7. **Compensation and Refund Rules**
   a. **Refund Processing**
      i. Inform the customer about the refund timeline based on the payment method.
      ii. If the refund is to the original payment method, then:
         - Process the refund.
            - Call the `process_refund` function.
      iii. Else, offer travel credits with a bonus percentage.
         - Apply the travel credit.
            - Call the `apply_travel_credit` function.
   b. **Compensation**
      i. Determine eligibility for compensation based on regional rules.
         - Call the `check_compensation_eligibility` function.
      ii. If eligible for meals or accommodation due to delays exceeding four hours, then:
         - Provide meal vouchers or hotel accommodation.
            - Call the `offer_accommodation` function.

8. **Special Cases**
   a. **Medical Emergencies**
    Ask the Customer for a medical certificate. 
    i. If a medical certificate is provided, then:
         - Allow full cancellation or offer flight credit.
            - Call the `process_flexible_cancellation` function.
      ii. If the customer or immediate family member is ill, then:
         - Waive change fees.
            - Call the `waive_change_fees` function.
   b. **Bereavement Cases**
      i. Offer flexibility on cancellations or changes.
      ii. Request proper bereavement documentation.
         - Call the `apply_bereavement_flexibility` function.
   c. **Group Bookings**
      i. Allow individual cancellations within a group booking.
         - Call the `process_partial_group_cancellation` function.
      ii. Permit one name change per passenger at no additional cost if made 7+ days before departure.
         - Call the `permit_name_change` function.
   d. **Unaccompanied Minors**
      i. Ensure proper supervision is arranged for flight changes.
         - Call the `arrange_supervision` function.
      ii. Prioritize rebooking for unaccompanied minors on the next available flight.
         - Call the `prioritize_minors_rebooking` function.

9. **FAQs for Common Scenarios**
   a. **Connecting Flight Affected**
      - Always prioritize rebooking for missed connections caused by airline delays.
         - Call the `prioritize_missed_connections` function.
   b. **Handling Upgrade Requests After Cancellation**
      - Offer upgrades based on availability.
         - Call the `offer_available_upgrades` function.
      - If unavailable, advise on fare differences.
   c. **Changing Destination**
      - Allow changes to a different destination.
      - Advise on applicable fare differences and change fees.
         - Call the `process_destination_change` function.
   d. **Third-Party Bookings**
      - Request the customer to contact the original booking agent.
      - Provide assistance if eligible for direct airline handling.
         - Call the `assist_third_party_booking` function.
   e. **Compensation for Inconvenience**
      - Assess eligibility based on the airline's compensation policy.
         - Call the `assess_compensation_request` function.
      - Inform the customer about available compensation options.
   f. **Preferred Flight Fully Booked**
      - Offer the next available option.
      - Place the customer on a waitlist for their preferred flight.
      - Offer an upgrade to a higher class if available.
         - Call the `offer_available_upgrades` function.

10. **Offer Further Assistance**
    a. Ask the customer if there is anything more you can assist with.
       - Call the `ask_clarification` function with the prompt: "Is there anything else I can assist you with today?"

11. **Case Resolution**
    a. Finalize and close the customer case with all resolution details.
       - Call the `case_resolution` function.
```

### Data quality check

We'll check that `o1-mini` has followed our instructions and used only the functions we provided.

If it added additional ones, we'll need to write functions for them before proceeding.

In [10]:
import re
from collections import Counter

def compare_lists(a, b):
    # Create Counters for both lists
    counter_a = Counter(a)
    counter_b = Counter(b)
    
    # Find elements that are only in A or have a higher count in A
    diff_a = counter_a - counter_b
    # Find elements that are only in B or have a higher count in B
    diff_b = counter_b - counter_a
    
    # Report differences
    print("Items in A that are not in B (or more of in A):")
    for item, count in diff_a.items():
        print(f"{item}: {count}")

    print("\nItems in B that are not in A (or more of in B):")
    for item, count in diff_b.items():
        print(f"{item}: {count}")

def extract_function_names(text):
    # Use regex to match text between backticks and extract function names
    pattern = r'`(.*?)`'
    matches = re.findall(pattern, text)
    return matches

# Extract a unique list of the functions in the generated routine
function_names_from_o1 = set(extract_function_names(flight_cancellation_routine))
# Extract a unique list of the functions in the generated routine
function_names_from_o1 = set(extract_function_names(flight_cancellation_routine))

# Extract the list of TOOLs we provided the model
function_names_defined = [tool["function"]["name"] for tool in TOOLS if tool["type"] == "function"]

# Print the differences
## Items that are in A and not in B need functions written for them
## Items that are in B and not in A are fine, they are just unused in the routine the model has written
compare_lists(function_names_from_o1, function_names_defined)

Items in A that are not in B (or more of in A):
: 1

Items in B that are not in A (or more of in B):
determine_refund_method: 1


## Step 2: Evaluation

Now that we have a routine generated with o1, we can run it against our evaluation suite and measure its accuracy.

We'll start by creating an agent that is equipped with the policy and a list of tools. It will be given messages from an existing conversation and will be tasked with determining the next best action to take

In [11]:
def agent_response(transcript, policy, model):
    try:
        messages = [
            {
                "role": "system",
                "content": f"""
You are a customer service agent that is responsible for handling airline related issues. Below is the exact policy that you must follow to address the customer's issue.

POLICY:
{policy}
                """
            }
        ]

        messages.extend(transcript)
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=TOOLS,
            parallel_tool_calls=False,
        temperature=0)
        
        return response.choices[0].message 
    except Exception as e:
        print(f"An error occurred: {e}")
        print(messages)

#### Function to execute the SQL query generated during the Chat process

In [12]:
def ask_database(conn, query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(dict(conn.execute(query).fetchone()))
    except Exception as e:
        results = None
    return results

#### Processing each row of data from the Evaluation dataset through the chat process and logging the actual final function call of the Chat

In [25]:
import math 

def safe_str(value):
    return "" if (isinstance(value, float) and math.isnan(value)) else str(value)

def process_row(row_number,test_row,policy,model,iteration=0, verbose=False):
    # Extract variables from test_row
    context = test_row['context']
    request = test_row['request']
    full_name = test_row['full_name']
    booking_ref = test_row['booking_ref']
    flight_no = test_row['flight_no']
    ticket_type = test_row['ticket_type']
    fare_rules = safe_str(test_row['fare_rules'])
    medical_certificate = safe_str(test_row['medical_certificate'])
    refund_amount = safe_str(test_row['refund_amount'])
    expected_function = test_row['expected_function']
    expected_inputs_str = test_row['expected_inputs']
    expected_inputs = json.loads(expected_inputs_str)

    CUSTOMER_PROMPT = f"""<objective>You are a customer named {full_name} trying to resolve an issue with a customer service agent.
    Continue providing the information the agent requests so they can solve your issue.  If you missed your connecting flight, be specific whether you want to
    address compensation or getting rescheduled to another flight.  If you are provided with a proposed flight change, confirm if you are ok or not with the change.
    If you have the information in your *details* then use that. Otherwise, generate an appropriate answer.</objective>
    <details>
    Booking Reference: {booking_ref}
    Flight Number: {flight_no}
    {f"Medical Reference: {medical_certificate}" if medical_certificate else ""}
    Context: {context}
    Your initial request: {request}
    </details>
    <guidance>
    Remember that YOU are the customer.
    If the agent needs some information, that must come from you. 
    Do not ask the agent for information.
    </guidance>
    """

    # Initialize transcript
    transcript = [{"role": "user", "content": request}]
    if verbose:
        print(f'Starting request: {request}')
    
    # Initialize variables
    is_correct = False

    loop_count = 0
    while True:
        loop_count += 1
        # print("transcript:", transcript)
        # input()
        if loop_count > 10:
            actual_function = None
            actual_inputs = None
            break
        # Call agent_response with the current transcript, policy, model
        response = agent_response(transcript, policy, GPT_MODEL)
        
        # Extract assistant message content and tool_calls
        if response:
            assistant_message_content = response.content
            tool_calls = response.tool_calls
        else:
            assistant_message_content = None
            tool_calls = None
        
        if not tool_calls:
            # Append assistant's message to transcript
            transcript.append({"role": "assistant", "content": assistant_message_content})
            
            # Get customer response
            customer_messages = [{"role": "system", "content": CUSTOMER_PROMPT}]
            customer_messages.extend(transcript)
            
            customer_response = client.chat.completions.create(model=GPT_MODEL, messages=customer_messages)
            
            # Append customer response to transcript
            transcript.append({"role": "user", "content": customer_response.choices[0].message.content})
            
            # Continue the loop
            continue
        else:
            # There is a tool call
            tool_call = tool_calls[0]  # Assuming we're only interested in the first tool call
            function_name = tool_call.function.name
            arguments = json.loads(tool_call.function.arguments)

            # If true the model will return the name of the tool / function to call and the argument(s)  
            # There is a tool call
            tool_call = tool_calls[0]  # Assuming we're only interested in the first tool call        
            tool_call_id = tool_call.id
            tool_function_name = tool_call.function.name

            if function_name == 'ask_clarification':
                # Take the 'prompt' from arguments and use it as assistant message
                clarification_prompt = arguments.get('prompt')
                
                # Append assistant's clarification message
                transcript.append({"role": "assistant", "content": clarification_prompt})
                
                # Get customer response
                customer_messages = [{"role": "system", "content": CUSTOMER_PROMPT}]
                customer_messages.extend(transcript)
                
                customer_response = client.chat.completions.create(model=GPT_MODEL, messages=customer_messages)
                
                # Append customer response to transcript
                transcript.append({"role": "user", "content": customer_response.choices[0].message.content})
                
                # Continue the loop
                continue
            elif function_name in ('verify_identity', 'check_ticket_type', 'check_fare_rules'):
                # Append assistant's message to transcript
                transcript.append(response)
                
                # Simulate the tool response
                tool_query_string = json.loads(tool_call.function.arguments)['query']
                conn = sqlite3.connect(r'FlightBooking.db') 
                conn.row_factory = sqlite3.Row
                results = ask_database(conn, tool_query_string)
                conn.close()
                # tool_response = f'Ticket type: {results}'
                if results:
                    if function_name == 'verify_identity':
                        tool_response = "True"
                    else:
                        tool_response = results
                else:
                    tool_response = "False"
                # Append the tool response to transcript
                transcript.append({"role": "tool", "content": tool_response, "tool_call_id": tool_call.id})
                
                # Continue the loop
                continue
            elif function_name == 'rebook_without_fee':
                # Take the 'prompt' from arguments and use it as assistant message
                clarification_prompt = "Could you please confirm if you are ok with the rebooking"
                
                # Append assistant's clarification message
                transcript.append({"role": "assistant", "content": clarification_prompt})
                
                # Get customer response
                customer_messages = [{"role": "system", "content": CUSTOMER_PROMPT}]
                customer_messages.extend(transcript)
                
                customer_response = client.chat.completions.create(model=GPT_MODEL, messages=customer_messages)
                
                # Append customer response to transcript
                transcript.append({"role": "user", "content": customer_response.choices[0].message.content})
                
                # Continue the loop
                continue
            elif function_name == 'check_next_available_flight':
                # Append assistant's message to transcript
                transcript.append(response)
                
                # simulating the tool response
                tool_response = f'Next available flight: LMG091 at 10:00 AM Tomorrow'
                
                # Append the tool response to transcript
                transcript.append({"role": "tool", "content": tool_response, "tool_call_id": tool_call.id})
                
                # Continue the loop
                continue
            elif function_name == 'provide_alternative_options':
                # Append assistant's message to transcript
                transcript.append(response)
                
                # simulating the tool response
                tool_response = f'Next available flight: FR9876 at 5:00 PM Tomorrow'
                
                # Append the tool response to transcript
                transcript.append({"role": "tool", "content": tool_response, "tool_call_id": tool_call.id})
                
                # Continue the loop
                continue
            else:
                if verbose:
                    print('Got a successful function call')
                # Stop and compare function call and arguments with expected_function and expected_inputs
                transcript.append(response)
                actual_function = function_name
                actual_inputs = arguments
                is_correct = (actual_function == expected_function) and (str(actual_inputs) == str(expected_inputs))
                
                # Break the loop
                break
    return {
        'request': request,
        'context': context,
        'expected_function': expected_function,
        'expected_inputs': expected_inputs,
        'actual_function': actual_function,
        'actual_inputs': actual_inputs,
        'is_correct': is_correct,
        'transcript': transcript
    }

# added post-filming to address transcript size
def filter_messages(message_list):
    messages = []
    for item in message_list:
        # Convert the item to a string and check if 'ChatCompletionMessage' is in it.
        if 'ChatCompletionMessage' in str(item):
            #print(item)
            message = {"role": "assistant",
                       "tool_call": {
                           "name": item.tool_calls[0].function.name,
                           "arguments": item.tool_calls[0].function.arguments
                       }
                      }
            messages.append(message)
        else:
            messages.append(item)
    return messages

#### Function to evaluate the test cases on a parallel basis - Free account tiers run into rate limit errors.  use only if on chatgpt plus or pro account

In [101]:
# def evaluate_function_calls(df, policy, model, i=0, verbose=False):
#     records = []

#     # Use ThreadPoolExecutor to process rows in parallel
#     with ThreadPoolExecutor() as executor:
#         futures = {executor.submit(process_row, row_number, row, policy, model, i, verbose): row_number for row_number, row in df.iterrows()}
#         for future in futures:
#             record = future.result()
#             records.append(record)
    
#     df = pd.DataFrame(records)
#     df['cleaned_transcript'] = df['transcript'].apply(filter_messages)
#     total_accuracy = df['is_correct'].mean()
#     return df, total_accuracy

#### Function to pass each row of the evaluation test data through the Chat evaluation function

In [30]:
def evaluate_function_calls(df, policy, model, i=0, verbose=False):
    records = []
    for row_number, row in df.iterrows():
        future = process_row(row_number,row,policy,model,i, verbose)
        print ("processed row:", row_number)
        records.append(future)

    df = pd.DataFrame(records)
    df['cleaned_transcript'] = df['transcript'].apply(filter_messages)
    total_accuracy = df['is_correct'].mean()
    return df, total_accuracy

In [31]:
eval_df = pd.read_csv('evals/policyEvals.csv')

In [32]:
df, accuracy = evaluate_function_calls(eval_df, flight_cancellation_routine, GPT_MODEL, verbose=False)

processed row: 0
processed row: 1
processed row: 2
processed row: 3
processed row: 4
processed row: 5
processed row: 6
processed row: 7
processed row: 8
processed row: 9
processed row: 10
processed row: 11
processed row: 12
processed row: 13
processed row: 14
processed row: 15
processed row: 16


#### Display the Accuracy of the Test results and the details of the Test 

In [33]:
# Display the accuracy as a mini header
display(Markdown(f"### Accuracy: {accuracy:.2%}"))

display(df)

### Accuracy: 88.24%

,request,context,expected_function,expected_inputs,actual_function,actual_inputs,is_correct,transcript,cleaned_transcript
0,"I would like to cancel my flight, please.",I want to cancel my flight as I am initiating the cancellation myself.,process_partial_refund,"{'booking_reference': 'ABC123', 'refund_amount': 198.0}",process_partial_refund,"{'booking_reference': 'ABC123', 'refund_amount': 198.0}",True,"[{'role': 'user', 'content': 'I would like to cancel my flight, please.'}, {'role': 'assistant', 'content': 'Could you please provide me with your booking reference, full name, and flight number so I can assist you further?'}, {'role': 'user', 'content': 'My booking reference is ABC123, my name is John Doe, and the flight number is XYZ789.'}, ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_CI5sBh24cKCiRgZOTh1EbuRJ', function=Function(arguments='{""query"":""SELECT * FROM bookinginfo WHERE booking_ref = \'ABC123\' AND full_name = \'John Doe\' AND flight_no = \'XYZ789\'""}', name='verify_identity'), type='function')]), {'role': 'tool', 'content': 'True', 'tool_call_id': 'call_CI5sBh24cKCiRgZOTh1EbuRJ'}, {'role': 'assistant', 'content': 'Could you please clarify if you would like to cancel your booking, make changes to your flight, or inquire about compensation?'}, {'role': 'user', 'content': 'I would like to proceed with cancelling my flight.'}, ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_goWnyydAdl5nmfocA2Qk4fqJ', function=Function(arguments='{""query"":""SELECT ticket_type FROM bookinginfo WHERE booking_ref = \'ABC123\'""}', name='check_ticket_type'), type='function')]), {'role': 'tool', 'content': '{'ticket_type': 'refundable'}', 'tool_call_id': 'call_goWnyydAdl5nmfocA2Qk4fqJ'}, ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_kHsN85eKigvRWHqicuhdk64U', function=Function(arguments='{""query"":""SELECT fare_rules, refund_amount FROM bookinginfo WHERE booking_ref = \'ABC123\'""}', name='check_fare_rules'), type='function')]), {'role': 'tool', 'content': '{'fare_rules': 'partial_refund', 'refund_amount': 198.0}', 'tool_call_id': 'call_kHsN85eKigvRWHqicuhdk64U'}, ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_MvtTIpwcBuof6NUfjf553bJl', function=Function(arguments='{""booking_reference"":""ABC123"",""refund_amount"":198.0}', name='process_partial_refund'), type='function')])]","[{'role': 'user', 'content': 'I would like to cancel my flight, please.'}, {'role': 'assistant', 'content': 'Could you please provide me with your booking reference, full name, and flight number so I can assist you further?'}, {'role': 'user', 'content': 'My booking reference is ABC123, my name is John Doe, and the flight number is XYZ789.'}, {'role': 'assistant', 'tool_call': {'name': 'verify_identity', 'arguments': '{""query"":""SELECT * FROM bookinginfo WHERE booking_ref = 'ABC123' AND full_name = 'John Doe' AND flight_no = 'XYZ789'""}'}}, {'role': 'tool', 'content': 'True', 'tool_call_id': 'call_CI5sBh24cKCiRgZOTh1EbuRJ'}, {'role': 'assistant', 'content': 'Could you please clarify if you would like to cancel your booking, make changes to your flight, or inquire about compensation?'}, {'role': 'user', 'content': 'I would like to proceed with cancelling my flight.'}, {'role': 'assistant', 'tool_call': {'name': 'check_ticket_type', 'arguments': '{""query"":""SELECT ticket_type FROM bookinginfo WHERE booking_ref = 'ABC123'""}'}}, {'role': 'tool', 'content': '{'ticket_type': 'refundable'}', 'tool_call_id': 'call_goWnyydAdl5nmfocA2Qk4fqJ'}, {'role': 'assistant', 'tool_call': {'name': 'check_fare_rules', 'arguments': '{""query"":""SELECT fare_rules, ref

#### Display the Accuracy of the Test results and the details of the Test - Another test gave a slightly different combo of results but accuracy still at 88.24%

In [56]:
# Display the accuracy as a mini header
display(Markdown(f"### Accuracy: {accuracy:.2%}"))

display(df)

### Accuracy: 88.24%

request  \
0                                   I would like to cancel my flight, please.   
1                I'd like to initiate a change to my flight for a later date.   
2                 My flight was canceled by the airline, and I want a refund.   
3                                  I want to cancel my non-refundable ticket.   
4                                      I want to cancel my refundable ticket.   
5                 I missed my connecting flight because of the airline delay.   
6   I want to know if I am eligible for compensation because of a long delay.   
7                                           I would like to rebook my flight.   
8                  I want to cancel my flight because of a medical emergency.   
9              I would like to change the passenger name on my group booking.   
10          The airline canceled my flight, and I need a hotel for the night.   
11                         I need to change my flight within the next 7 days.   
12               I want to change my non-flexible flight to a different date.   
13         The airline canceled my flight due to weather. Can I get rebooked?   
14                              I need a refund for my non-refundable flight.   
15                     I need a refund for a flight I booked a few hours ago.   
16      I missed my connecting flight because of a delay. Can I get rebooked?   

                                                                                                                context  \
0                                                I want to cancel my flight as I am initiating the cancellation myself.   
1                                                                       I want to change my flight to a different date.   
2                              I want to request a refund because my flight was canceled by the airline due to weather.   
3                                                          I want to cancel my flight but I think it is non-refundable.   
4                                                            I want to cancel my flight and I believe it is refundable.   
5                     I want to address the issue of missing my connecting flight due to a delay caused by the airline.   
6                                        I want to know if I can receive compensation due to a long delay on my flight.   
7                                          I want to rebook my flight, and I have provided my details for verification.   
8                                                                I want to cancel my flight due to a medical emergency.   
9                                               I want to change the name of one of the passengers in my group booking.   
10                                                I want to find a hotel because my flight was canceled by the airline.   
11          I want to change my flight because it's within the next week and I hope I can do it without any extra fees.   
12                       I want to change my flight date, but I know my ticket is non-flexible which might incur a fee.   
13                             I want to get rebooked on a flight since my original flight was canceled due to weather.   
14                                                         I want to cancel my flight but I think it is non-refundable.   
15  I want to get a refund for a recently booked flight and I have provided the necessary details to verify my booking.   
16                       I want to get rebooked on another flight because I missed my connecting flight due to a delay.   

                 expected_function  \
0           process_partial_refund   
1                 apply_change_fee   
2              process_full_refund   
3              offer_flight_credit   
4           process_partial_refund   
5    prioritize_missed_connections   
6   check_compensation_eligibility   
7            process_change_no_fee   
8    process_flexible_cancellation   
9               

0.8823529411764706